In [15]:
import os.path
import re

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [16]:
import os

my_folder = "../data/pdfs/ROCNE SPRAVY 2020" # your path here
count = 0

pdf_count=0
xls_count=0
doc_count=0
others=0

for root, dirs, files in os.walk(my_folder):
    # if os.path.basename(root) == "CERI":
        # print((root, dirs, files))
    for f in files:
        if f.lower().endswith(".pdf") :
            pdf_count+=1
        elif f.lower().endswith(".xls") or f.lower().endswith(".xlsx"):
            xls_count+=1
            print(f)
        elif f.lower().endswith(".doc") or f.lower().endswith(".docx"):
            doc_count+=1
            print(f)
        else:
            others+=1


print((pdf_count, xls_count, doc_count, others))

    #count += len([fn for fn in files if fn.endswith(".pdf")])
#print(count)

(0, 0, 0, 0)


In [17]:
# html = open('../data/pdfs/ROCNE SPRAVY  2021/ACROSS FUNDING správa 2021.xhtml').read()
html = open('../../../data/pdfs/ROCNE SPRAVY  2021/Alpha Quest Funds SICAV plc správa 2021.xhtml').read()
soup = BeautifulSoup(html)
[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title', 'img'])]
all_content = soup.find('div', attrs={'id': "page-container"})
visible_text = soup.getText()
print(visible_text)

﻿












Statement of Financial Position1 Reporting entity2 Basis of preparation2.2 Basis of measurement2.3 Functional and presentation currency2.4 Use of assumptions, estimates and judgements2.5 New standards and interpretations not yet adopted3.1 Foreign currency transactions3.2 Interest income and interest expense3.3 Fees and commission expenses3.4 Restitution points3.5 Financial assets and financial liabilities3.5.1 Recognition and initial measurement3 Significant accounting policies (continued)3.5 Financial assets and financial liabilities (continued)3 Significant accounting policies (continued)3.5 Financial assets and financial liabilities (continued)3 Significant accounting policies (continued)3.5 Financial assets and financial liabilities (continued)3.5.8 OffsettingFinancial assets and financial liabilities are offset and the net amount presented in the statement of financial position when, and only when, the Company has currently a legally enforceable right to offset the 

In [18]:
def printTable(myDict, colList=None, sep='\uFFFA'):
   """ Pretty print a list of dictionaries (myDict) as a dynamically sized table.
   If column names (colList) aren't specified, they will show in random order.
   sep: row separator. Ex: sep='\n' on Linux. Default: dummy to not split line.
   Author: Thierry Husson - Use it as you want but don't blame me.
   """
   if not colList: colList = list(myDict[0].keys() if myDict else [])
   myList = [colList] # 1st row = header
   for item in myDict: myList.append([str(item[col] or '') for col in colList])
   colSize = [max(map(len,(sep.join(col)).split(sep))) for col in zip(*myList)]
   formatStr = ' | '.join(["{{:<{}}}".format(i) for i in colSize])
   line = formatStr.replace(' | ','-+-').format(*['-' * i for i in colSize])
   item=myList.pop(0); lineDone=False
   while myList or any(item):
      if all(not i for i in item):
         item=myList.pop(0)
         if line and (sep!='\uFFFA' or not lineDone): print(line); lineDone=True
      row = [i.split(sep,1) for i in item]
      print(formatStr.format(*[i[0] for i in row]))
      item = [i[1] if len(i)>1 else '' for i in row]

In [19]:
print(len(all_content.findAll('div', attrs={'id': re.compile("(pf*)"),
                                     'class': 'pf'})))

def get_class_meta_from_div(tag, class_meta_name: str):
    ret_list = [class_meta.split(class_meta_name)[1] for class_meta in tag['class'] if class_meta_name in class_meta]
    return ret_list[0] if len(ret_list)>0 else None

texts = {}
class_meta_names = ["x", "y", "ff", "fs", "fc"]
text_count = -1
for i, page in enumerate(all_content.findAll('div', attrs={'id': re.compile("(pf*)")})):

    page_content = page.find('div', attrs={'class': 'pc'})
    for j, div in enumerate(page_content.findAll('div')):

        div_text = div.getText().strip()

        if not div_text or all(not x.isalpha() for x in div_text):
            continue

        meta_info = {}
        #print(div['class'])
        other_classes = [cls for cls in div['class'] if not cls.startswith(tuple(class_meta_names))]
        # print(other_classes)
        for class_meta_name in class_meta_names:
            meta_info[class_meta_name] = get_class_meta_from_div(div, class_meta_name)
            # print(meta_info[class_meta_name])
        if j>0 and text_count in texts.keys()\
           and np.array_equal(other_classes, texts[text_count]["other_classes"])\
           and meta_info["ff"] == texts[text_count]["ff"]\
           and meta_info["fs"] == texts[text_count]["fs"]:
            texts[text_count]["text"]+=div_text
        else:
            text_count+=1
            texts[text_count]={}
            texts[text_count]["text"]=div_text

        texts[text_count]['page'] = i
        texts[text_count]["other_classes"] = [cls for cls in div['class'] if not cls.startswith(tuple(class_meta_names))]
        for class_meta_name in class_meta_names:
            texts[text_count][class_meta_name] = get_class_meta_from_div(div, class_meta_name)

doc_df = pd.DataFrame.from_dict(texts, orient='index')

display(doc_df)

63


,text,page,other_classes,x,y,ff,fs,fc
0,ALPHA QUEST FUNDS SICAV p.l.c.,0,"[t, m0, h4, sc0, ls0, ws0]",3,13,4,2,0
1,ANNUAL REPORT,0,"[t, m0, h6, sc0, ls3, ws2]",4,17,4,0,0
2,For the year ended 31 December 2021,0,"[t, m0, h6, sc0, ls4, ws3]",5,19,4,0,0
3,Company Registration Number: SV 430,0,"[t, m0, h5, sc0, ls0, ws0]",6,3b,2,3,0
4,ALPHA QUEST FUNDS SICAV p.l.c.,1,"[t, m0, h8, sc0, ls0, ws0]",1,3d,4,4,0
...,...,...,...,...,...,...,...,...
3274,"knowledge and belief, we require for the purpo...",62,"[t, m0, h2, sc0, ls4c, ws58]",114,bbb,1f,1,0
3275,The Partner authorised to sign on behalf of KP...,62,"[t, m0, h2, sc0, ls4c, ws120]",10b,bbd,1f,1,0
3276,independent auditors’ report is Noel Mizzi.,62,"[t, m0, h2, sc0, ls4c, ws58]",10b,bbe,1f,1,0
3277,KPMG 29 April 2022,62,"[t, m0, h2, sc0, ls1e, ws5b]",10b,bc3,22,1,6


In [20]:
for text in doc_df['text']:
    print(text)
    print("---------------")

ALPHA QUEST FUNDS SICAV p.l.c.
---------------
ANNUAL REPORT
---------------
For the year ended 31 December 2021
---------------
Company Registration Number: SV 430
---------------
ALPHA QUEST FUNDS SICAV p.l.c.
---------------
CONTENTS
---------------
Page
---------------
ANNUAL REPORT
---------------
Directors, Officers and other information
---------------
Directors, Officers and other information
---------------
Report of the Directors
---------------
Report of the Directors
---------------
Statement of Directors’ Responsibilities
---------------
Statement of Directors’ Responsibilities
---------------
Specific Disclosures
---------------
Specific Disclosures
---------------
Financial Statements:
---------------
Financial Statements:
---------------
Statement of Financial Position
---------------
Statement of Financial Position
---------------
Statement of Comprehensive Income
---------------
Statement of Comprehensive Income
---------------
Statement of Changes in Net Assets Attri

In [22]:
def get_param_stats(dataframe_elements, param_name):
    dataframe_elements['index1'] = dataframe_elements.index
    param_counts = [dataframe_elements[dataframe_elements[param_name] == param_val]["index1"].count() for param_val in dataframe_elements[param_name]]
    sum_of_all_texts = dataframe_elements["index1"].count()
    dataframe_elements[f'{param_name}_frequency'] = param_counts / sum_of_all_texts
    return dataframe_elements

doc_df = get_param_stats(doc_df, 'ff')
doc_df = get_param_stats(doc_df, 'fs')
doc_df = get_param_stats(doc_df, 'fc')

display(doc_df)
doc_df.to_csv("../data/out/test.csv")

# def store_base_file_stat(dataframe_elements, out_firm_dir):
#     dataframe_elements['index1'] = dataframe_elements.index
#     fs_counts = [dataframe_elements[dataframe_elements['fs'] == size]["index1"].count() for size in
#               dataframe_elements['fs']]
#     sum_of_all_texts = dataframe_elements["index1"].count()
#     dataframe_elements['size_frequency'] = counts / sum_of_all_texts
#     dataframe_elements.to_csv(os.path.join(out_firm_dir, "base_file_stat.csv"), index=False)
#
#     return dataframe_elements, os.path.join(out_firm_dir, "base_file_stat.csv")

,text,page,other_classes,x,y,ff,fs,fc,index1,ff_frequency,fs_frequency,fc_frequency
0,ALPHA QUEST FUNDS SICAV p.l.c.,0,"[t, m0, h4, sc0, ls0, ws0]",3,13,4,2,0,0,0.146386,0.010674,0.711192
1,ANNUAL REPORT,0,"[t, m0, h6, sc0, ls3, ws2]",4,17,4,0,0,1,0.146386,0.010674,0.711192
2,For the year ended 31 December 2021,0,"[t, m0, h6, sc0, ls4, ws3]",5,19,4,0,0,2,0.146386,0.010674,0.711192
3,Company Registration Number: SV 430,0,"[t, m0, h5, sc0, ls0, ws0]",6,3b,2,3,0,3,0.335163,0.182373,0.711192
4,ALPHA QUEST FUNDS SICAV p.l.c.,1,"[t, m0, h8, sc0, ls0, ws0]",1,3d,4,4,0,4,0.146386,0.018908,0.711192
...,...,...,...,...,...,...,...,...,...,...,...,...
3274,"knowledge and belief, we require for the purpo...",62,"[t, m0, h2, sc0, ls4c, ws58]",114,bbb,1f,1,0,3274,0.075328,0.061299,0.711192
3275,The Partner authorised to sign on behalf of KP...,62,"[t, m0, h2, sc0, ls4c, ws120]",10b,bbd,1f,1,0,3275,0.075328,0.061299,0.711192
3276,independent auditors’ report is Noel Mizzi.,62,"[t, m0, h2, sc0, ls4c, ws58]",10b,bbe,1f,1,0,3276,0.075328,0.061299,0.711192
3277,KPMG 29 April 2022,62,"[t, m0, h2, sc0, ls1e, ws5b]",10b,bc3,22,1,6,3277,0.007014,0.061299,0.012809


FileNotFoundError: [Errno 2] No such file or directory: '../data/out/test.csv'

In [ ]:
def store_unique_formatting_stats(dataframe_elements, out_firm_dir):
    dataframe_unique_formatting = dataframe_elements.groupby([ 'fs', 'ff', 'fs_frequency', 'ff_frequency']).agg({
        'text': list,
        "page": list,
        'index1': [list, 'count']})
    dataframe_unique_formatting.columns = ['_'.join(col).strip() for col in
                                           dataframe_unique_formatting.columns.values]
    dataframe_unique_formatting.reset_index(inplace=True)
    dataframe_unique_formatting.to_csv(os.path.join(out_firm_dir, "unique_formatting.csv"), index=True)

    return dataframe_unique_formatting, os.path.join(out_firm_dir, "unique_formatting.csv")

In [188]:
dataframe_unique_formatting, dataframe_unique_formatting_csv_path = store_unique_formatting_stats(doc_df, "../data/out/")

In [189]:
def give_label(row):
    if row['fs_frequency'] == 0 or row['ff_frequency'] == 0:
        return "other"
    elif row['fs_frequency'] < 0.25 and row['ff_frequency'] < 0.25:
        return "header"
    elif row['fs_frequency'] < 0.15:
        return "header"
    elif row['fs_frequency'] >= 0.15:
        return "text"
    # zvysok by mal byt trash -- nadpisy v tabulkach, chyby pri analyze pdf ...
    return None

dataframe_unique_formatting['label'] = dataframe_unique_formatting.apply(give_label, axis=1)
dataframe_unique_formatting.to_csv(dataframe_unique_formatting_csv_path, index=True)
doc_df['label'] = doc_df.apply(give_label, axis=1)
doc_df.to_csv("../data/out/test.csv")

display(doc_df)

,text,page,other_classes,x,y,ff,fs,fc,index1,ff_frequency,fs_frequency,fc_frequency,label
0,ALPHA QUEST FUNDS SICAV p.l.c.,0,"[t, m0, h4, sc0, ls0, ws0]",3,13,4,2,0,0,0.146386,0.010674,0.711192,header
1,ANNUAL REPORT,0,"[t, m0, h6, sc0, ls3, ws2]",4,17,4,0,0,1,0.146386,0.010674,0.711192,header
2,For the year ended 31 December 2021,0,"[t, m0, h6, sc0, ls4, ws3]",5,19,4,0,0,2,0.146386,0.010674,0.711192,header
3,Company Registration Number: SV 430,0,"[t, m0, h5, sc0, ls0, ws0]",6,3b,2,3,0,3,0.335163,0.182373,0.711192,text
4,ALPHA QUEST FUNDS SICAV p.l.c.,1,"[t, m0, h8, sc0, ls0, ws0]",1,3d,4,4,0,4,0.146386,0.018908,0.711192,header
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,"knowledge and belief, we require for the purpo...",62,"[t, m0, h2, sc0, ls4c, ws58]",114,bbb,1f,1,0,3274,0.075328,0.061299,0.711192,header
3275,The Partner authorised to sign on behalf of KP...,62,"[t, m0, h2, sc0, ls4c, ws120]",10b,bbd,1f,1,0,3275,0.075328,0.061299,0.711192,header
3276,independent auditors’ report is Noel Mizzi.,62,"[t, m0, h2, sc0, ls4c, ws58]",10b,bbe,1f,1,0,3276,0.075328,0.061299,0.711192,header
3277,KPMG 29 April 2022,62,"[t, m0, h2, sc0, ls1e, ws5b]",10b,bc3,22,1,6,3277,0.007014,0.061299,0.012809,header


In [190]:
last_header = ''
headers = []
for text, label in zip(doc_df.text, doc_df.label):
    if label == 'header':
        last_header = text
    headers.append(last_header)
doc_df['belonging_header'] = headers
doc_df.to_csv("../data/out/test.csv")

In [191]:
def get_number_of_words(row):
    return len(row['text'].split())

def get_number_of_chars(row):
    return len(row['text'])

def get_number_of_alphachars(row):
    return sum(c.isalpha() for c in row['text'])

def store_text_info_only(dataframe_elements, out_firm_dir):
    mask = (dataframe_elements.label == 'text')
    dataframe_texts = dataframe_elements[mask].copy()
    dataframe_texts['words'] = dataframe_texts.apply(get_number_of_words, axis=1)
    dataframe_texts['chars'] = dataframe_texts.apply(get_number_of_chars, axis=1)
    dataframe_texts['alpha'] = dataframe_texts.apply(get_number_of_alphachars, axis=1)
    dataframe_texts.to_csv(os.path.join(out_firm_dir, "all_texts_info.csv"))

    return dataframe_texts, os.path.join(out_firm_dir, "all_texts_info.csv")

dataframe_texts, dataframe_texts_csv_path = store_text_info_only(doc_df, "../data/out/")

In [192]:
def filter_texts(dataframe_texts, out_firm_dir):
    mask2 = (dataframe_texts.alpha > 40) & (dataframe_texts.words > 3)
    dataframe_texts_clean = dataframe_texts[mask2].copy()
    dataframe_texts_clean['d'] = 0
    dataframe_texts_clean['h'] = 0
    dataframe_texts_clean.to_csv(os.path.join(out_firm_dir, "filtered_texts_info.csv"))
    return dataframe_texts_clean, os.path.join(out_firm_dir, "filtered_texts_info.csv")
dataframe_texts_clean, dataframe_texts_clean_csv_path = filter_texts(dataframe_texts, "../data/out/")